In [11]:
import pandas as pd
import json
from pathlib import Path
from Bio import SeqIO
from Bio.Seq import Seq

In [52]:
# -- global --

SUB_MATRICES_PATH='../utils/substitution_matrices.json'
data=json.load(open(SUB_MATRICES_PATH))
SUB_MATRICES = {k:data[k]['type'] for k in list(data.keys())}

# -- user defined variables: --

# mandatory
FASTA_1 = '../../example/protein_seq1.fasta'
FASTA_2 = '../../example/protein_seq2.fasta'

# optional
SEQ_TYPE='protein'
WINDOW_SIZE=10
THRESHOLD=23
SCORE_MATRIX='blosum62'



In [ ]:
# ----------------------- mandatory -----------------------

In [41]:
def validate_path(path_str):
    path=Path(path_str)
    if not path.exists():
        print(f'<!> Path {path} does not exist.')
        return False
    return True

validate_path(FASTA_2)


True

In [42]:
def is_protein(fasta_file):
    try:
        seq = SeqIO.read(fasta_file, 'fasta')
        seq = str(seq.seq)
        if all([aa in 'ACDEFGHIKLMNPQRSTVWY' for aa in seq]):
            return True
        else:
            return False
    except:
        print(f'<!> File {fasta_file} is not a valid fasta file.')
        return False
    
def is_dna(fasta_file):
    try:
        seq = SeqIO.read(fasta_file, 'fasta')
        seq = str(seq.seq)
        if all([nt in 'ACGT' for nt in seq]):
            return True
        else:
            return False
    except:
        print(f'<!> File {fasta_file} is not a valid fasta file.')
        return False
    
def get_type(fasta_file):
    if is_protein(fasta_file):
        return 'protein'
    elif is_dna(fasta_file):
        return 'dna'
    else:
        return None
    
SEQ_TYPE = get_type(FASTA_1)

In [ ]:
def get_seq_from_fasta(fasta_file):
    t= get_type(fasta_file)
    seq = SeqIO.read(fasta_file, 'fasta')
    seq.id = seq.id + f' {t}'
    return seq

SEQ_1 = get_seq_from_fasta(FASTA_1)
SEQ_2 = get_seq_from_fasta(FASTA_2)


In [ ]:
# ------- optional parameters -------

In [3]:
def validate_substitution_matrix(matrix_name='blosum62', sequence_type='protein'):
    types=['protein','dna']
    matrix_name = matrix_name.lower()
    sequence_type = sequence_type.lower()


    if sequence_type not in types:
        print('<!> Invalid sequence type. Must be one of: {}\n\t-- Taking protein as default--'.format(types))
        sequence_type='protein'

    keys = [k for k,v in SUB_MATRICES.items() if v==sequence_type]
    # print(keys)
    
    if matrix_name not in keys:
        if sequence_type=='protein':
            print('<!> Invalid substitution matrix. Must be one of: {}\n\t'.format(keys))
            matrix_name='blosum62'
        else:
            print('<!> Invalid substitution matrix. Must be one of: {}'.format(keys))
            matrix_name='DNAFull'
        
    print(f'\n\t -- Taking {matrix_name} as substitution matrix for {sequence_type} sequences --')
    return matrix_name, sequence_type


In [4]:
def read_submat_from_json(matrix_name, json_file=SUB_MATRICES_PATH):

    json_file = open(json_file)
    data = json.load(json_file)
    df = pd.DataFrame(data[matrix_name]['matrix'])
    df.index = df.columns
    return df

In [50]:
def validate_threshold(w=WINDOW_SIZE,t=THRESHOLD):
    # not using it for now 
    if t>w:
        print(f'<!> Threshold ({t}) must be less than window size ({w}).\n\t-- Taking threshold as half of window size --')
        t = w//2
    return t

In [51]:
def get_parameters(w=WINDOW_SIZE, t=THRESHOLD, sm=SCORE_MATRIX, stype=SEQ_TYPE, sub_mat_path=SUB_MATRICES_PATH):
    SCORE_MATRIX, SEQ_TYPE=validate_substitution_matrix(sm, stype)
    # THRESHOLD=validate_threshold(w, t)
    SUBSTITUTION_MATRIX=read_submat_from_json(sm, sub_mat_path)

    return WINDOW_SIZE, THRESHOLD, SCORE_MATRIX, SEQ_TYPE, SUBSTITUTION_MATRIX

WINDOW_SIZE, THRESHOLD, SCORE_MATRIX, SEQ_TYPE, SUBSTITUTION_MATRIX = get_parameters(WINDOW_SIZE, THRESHOLD, SCORE_MATRIX, SEQ_TYPE)


	 -- Taking blosum62 as substitution matrix for protein sequences --


In [53]:
def get_parameters_description(seq_type=SEQ_TYPE, window_size=WINDOW_SIZE, threshold=THRESHOLD, score_matrix=SCORE_MATRIX):
    return f'''
    -- Parameters --
    Sequence type: {seq_type}
    Window size: {window_size}
    Threshold: {threshold}
    Score matrix: {score_matrix}
    '''

print(get_parameters_description())


    -- Parameters --
    Sequence type: protein
    Window size: 10
    Threshold: 23
    Score matrix: blosum62
    


In [46]:
SUBSTITUTION_MATRIX

,A,R,N,D,C,Q,E,G,H,I,...,P,S,T,W,Y,V,B,Z,X,*
A,4,-1,-2,-2,0,-1,-1,0,-2,-1,...,-1,1,0,-3,-2,0,-2,-1,0,-4
R,-1,5,0,-2,-3,1,0,-2,0,-3,...,-2,-1,-1,-3,-2,-3,-1,0,-1,-4
N,-2,0,6,1,-3,0,0,0,1,-3,...,-2,1,0,-4,-2,-3,3,0,-1,-4
D,-2,-2,1,6,-3,0,2,-1,-1,-3,...,-1,0,-1,-4,-3,-3,4,1,-1,-4
C,0,-3,-3,-3,9,-3,-4,-3,-3,-1,...,-3,-1,-1,-2,-2,-1,-3,-3,-2,-4
Q,-1,1,0,0,-3,5,2,-2,0,-3,...,-1,0,-1,-2,-1,-2,0,3,-1,-4
E,-1,0,0,2,-4,2,5,-2,0,-3,...,-1,0,-1,-3,-2,-2,1,4,-1,-4
G,0,-2,0,-1,-3,-2,-2,6,-2,-4,...,-2,0,-2,-2,-3,-3,-1,-2,-1,-4
H,-2,0,1,-1,-3,0,0,-2,8,-3,...,-2,-1,-2,-2,2,-3,0,0,-1,-4
I,-1,-3,-3,-3,-1,-3,-3,-4,-3,4,...,-3,-2,-1,-3,-1,3,-3,-3,-1,-4


In [ ]:

# def is_fasta(fasta_file):
#     try:
#         SeqIO.read(fasta_file, 'fasta')
#         return True
#     except:
#         print(f'<!> File {fasta_file} is not a valid fasta file.')
#         return False
    
# is_fasta(FASTA_2)

True

'protein'